# Log Parse D3

This page shows ...

#### Setup the environment

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import pycamhd.pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt
import requests
import itertools
import os.path
from numpngw import write_png

#### Get a list of remote CamHD files to process

In [2]:
filenames = []
with open('d3.txt') as f:
    for line in f:
        filenames.append(line.strip())

#### Get the timestamp from each video file

In [3]:
#only do this once
#movtimes = []
#for filename in filenames:
#    try:
#        movtimes.append(camhd.get_timestamp(filename))
#    except:
#        print(filename)

# now just read it from a file
movtimes = []
with open('d3_movtimes.csv', 'r') as csvfile:
    for line in csvfile:
        movtimes.append(line.split(',')[1].strip())

#### Write start times to file

In [4]:
#with open('d4_movtimes.csv', 'w') as csvfile:
#    for i, filename in enumerate(filenames):
#        csvfile.write('%s,%s\n' % (filename, movtimes[i]))

#### Get list of log files (from local)

In [5]:
#%%time
#logfiles = []
#for i,filename in enumerate(filenames):
#    logfile = '/data/camhda301_%s_%s.log' % (filename[96:102], filename[103:107])
#    if os.path.isfile(logfile):
#        logfiles.append(logfile)

#### Get list of log file (from remote)

In [6]:
#%%time
#logfiles = []
#for i,filename in enumerate(filenames):
#    logfile = '%s/camhda301_%s_%s.log' % (filename[0:83], filename[96:102], filename[103:107])
#    r = requests.head(logfile)
#    if r.status_code==200:
#        logfiles.append(logfile)
        #continue
#    else:
#        print(logfile)
#        break
        
    #logfile = '%s/camhda301_%s_%s%s.log' % (filename[0:83], filename[96:102], filename[103:105], str(int(filename[105:107])-15))
    #r = requests.head(logfile)
    #print(logfile)
    #break
    #if r.status_code==200:
    #    print(logfile)
    #    logfiles.append(logfile)
    #else:
    #    logfiles.append(None)

#### Read logtimes.csv

In [7]:
logfiles = []
logtimes = []
with open('logtimes.csv', 'r') as f:
    for line in f:
        logfiles.append(line.split(',')[0])
        logtimes.append(line.split(',')[1].strip())

#### Build d3_meta

In [8]:
d3_meta = []
for i, filename in enumerate(filenames):
    for j, logtime in enumerate(logtimes):
        if int(movtimes[i]) > (int(logtime)-60) and int(movtimes[i]) < (int(logtime)+60):
            d3_meta.append([filename, str(movtimes[i]), '/data/%s' % logfiles[j], str(logtime)])

#### Get approximate time of p2z1 and append to d4_meta

In [9]:
def get_p2z1(logfile):
    p2z1 = None
    with open(logfile, 'r') as f:
        for line in f:
            if 'pan=143' in line or 'pan=145' in line:
                for line in f:
                    if 'time' in line[0:4]:
                        p2z1 = line.split(' ')[1].strip()
                        return p2z1

In [10]:
for entry in d3_meta:
    p2z1 = get_p2z1(entry[2])
    entry.append('p2z1')
    entry.append(str(p2z1));

#### Write out d3_meta

In [11]:
with open('d3_meta.csv', 'w') as f:
    for entry in d3_meta:
        f.write('%s,%s,%s,%s,%s,%s\n' % (entry[0],entry[1],entry[2],entry[3],entry[4],entry[5]))

#### Set up Dask delayed PNG write function

In [13]:
from dask.multiprocessing import get
from dask import delayed, compute

@delayed
def delayed_write_png(entry):
    filename = entry[0]
    p2z1_frame = int(round((int(entry[5])-int(entry[1]))*29.97))
    frame_numbers = np.arange(p2z1_frame+30*2,p2z1_frame+30*2+30*8,15)
    frame_count = camhd.get_frame_count(filename)
    if max(frame_numbers) <= frame_count:
        for frame_number in frame_numbers:
            frame = camhd.get_frame(filename, frame_number, 'gray16le')
            png_file = "d3/%s_%05.0d.png" % (filename[84:], frame_number)
            write_png(png_file, frame)
    return 1

In [ ]:
dummy = []
for entry in d3_meta:
    dummy.append(delayed_write_png(entry))
test = compute(*dummy)    

In [16]:
len(test)

1772

In [ ]:
for entry in d3_meta:
    filename = entry[0]
    p2z1_frame = int(round((int(entry[5])-int(entry[1]))*29.97))
    frame_numbers = np.arange(p2z1_frame+30*2,p2z1_frame+30*2+30*8,15)
    frame_count = camhd.get_frame_count(filename)
    if max(frame_numbers) <= frame_count:
        for frame_number in frame_numbers:
            frame = camhd.get_frame(filename, frame_number, 'gray16le')
            png_file = "d3/%s_%05.0d.png" % (filename[84:], frame_number)
            write_png(png_file, frame)

In [ ]:
#plt.plot(frame_numbers)
trace = go.Scatter(y=logtimes-movtimes)
data = [trace]
py.iplot(data, filename='basic-line')

In [ ]:
%%time
from numpngw import write_png
for i,frame_number in enumerate(frame_numbers):
    frame_number = frame_number-60
    filename = d3_meta[i][0]
    frame = camhd.get_frame(filename, frame_number, 'gray')
    png_file = "%s_%s.png" % (filename[84:], frame_number)
    write_png(png_file, frame)

In [ ]:
(p2z0times[0]-movtimes[0])*29.97

In [ ]:
frame_numbers[0]

In [ ]:
#def get_p2z0(logfile):
#    p2z0 = ''
#    r = requests.get(logfile, stream=True)
#    for line in r.iter_lines():
#        if 'pan=143' in str(line):
#             for line in r.iter_lines():
#                    if 'STOPPED' in str(line):
#                        p2z0 = int(str(line).split(' ')[2].strip()[:-1])
#                        break
                        #return int(str(line).split(' ')[2].strip()[:-1])
#    return p2z0

In [ ]:
def get_p2z0(logfile):
    p2z0 = None
    #print(logfile)
    with open(logfile, 'r') as f:
        for line in f:
            #print(line)
            if 'pan=143' in line:
                #print(line)
                for line in f:
                    if 'STOPPED' in line:
                        #p2z0 = line
                        #print(line) 
                        p2z0 = int(line.split(' ')[2].strip()[:-1])
                        #return line.split(' ')[2].strip()[:-1]
                        #print(p2z0)
                        return p2z0
                        
                        
    #return p2z0

In [ ]:
for entry in d3_meta:
    #print(i)
    p2z1 = get_p2z1(entry[2])
    entry.append('p2z1')
    entry.append(str(p2z1));

In [ ]:
d3_meta[1]

In [ ]:
p2z0_frame = []
for entry in d3_meta:
    p2z0_frame.append(round((int(entry[5])-int(entry[1]))*29.97))

In [ ]:
trace = go.Scatter(y=p2z0_frame)
data = [trace]
py.iplot(data, filename='basic-line')

In [ ]:
with open('d3_meta.csv', 'w') as f:
    for entry in d3_meta:
        f.write('%s,%s,%s,%s,%s,%s\n' % (entry[0],entry[1],entry[2],entry[3],entry[4],entry[5]))

In [ ]:
d3_meta[907]

In [ ]:
movtimes = np.empty((1,0), dtype=np.int32)
logtimes = np.empty((1,0), dtype=np.int32)
p2z0times = np.empty((1,0), dtype=np.int32)
for entry in d3_meta:
    movtimes = np.append(movtimes, int(entry[1]))
    logtimes = np.append(logtimes, int(entry[3]))
    p2z0times = np.append(p2z0times, int(entry[5]))
   # movtimes.append(int(entry[1]))
    #logtimes.append(int(entry[3]))
    #p2z0times.append(int(entry[5]))

In [ ]:
def get_

In [ ]:
plt.ylim(0, 60000000)
plt.plot(logtimes-min(logtimes));

In [ ]:
trace1 = go.Scatter(y=logtimes-min(logtimes))
trace2 = go.Scatter(y=movtimes-min(movtimes))
trace3 = go.Scatter(y= p2z0times-movtimes)
data = [trace3]
py.iplot(data, filename='basic-line')

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np

N = 500
random_x = np.linspace(0, 1, N)
random_y = np.random.randn(N)

# Create a trace
trace = go.Scatter(
    x = random_x,
    y = random_y
)

data = [trace]

py.iplot(data, filename='basic-line')
dir(test)

In [ ]:
plt.ylim(40, 100)
plt.xlim(790, 800)
plt.plot(p2z0times-logtimes);

In [ ]:
d3_meta[796]

In [ ]:
mov_number = 700
frame_number = int(round((int(d3_meta[mov_number][5])-int(d3_meta[mov_number][1]))*29.97))+60
frame = camhd.get_frame(d3_meta[mov_number][0], frame_number, 'gray16le')
fig, ax = plt.subplots()
im1 = ax.imshow(frame)
im1.set_cmap('gray');
plt.title(frame_number)


In [ ]:
fig, ax = plt.subplots();
im1 = ax.imshow(frame);
im1.set_cmap('gray');

In [ ]:
for entry in d3_meta:
    p2z0 = get_p2z0('%s' % entry[2])
    #str(line).split(' ')[2].strip()[:-1]
    print(p2z0)
    #print(entry.append(3))
    #entry.append('p2z0')
    #entry.append(str(p2z0)
    #print(entry)
    #print(d3_meta[0])
    if p2z0==0:
        break

print(entry)

In [ ]:
print(entry)

In [ ]:
','.join(str(d3_meta[1])

In [ ]:
print(d3_meta[1000])

In [ ]:
p2z0 = []
for i, logfile in enumerate(logfiles):
    p2z0.append(get_p2z0(logfile))

In [ ]:
p2z0

In [ ]:
print(logfiles[1400])

In [ ]:
test = []
for i, logfile in enumerate(logfiles):
    r = requests.get(logfile, stream=True)
    for line in r.iter_lines():
        #print(line)
        if 'pan=143' in str(line):
            for line in r.iter_lines():
                #print(line)
                if 'STOPPED' in str(line):
                    #print(str(line).split(' ')[2].strip()[:-1])
                    test.append(str(line).split(' ')[2].strip()[:-1])
                    break
                #break
            #print(next(itertools.islice(r.iter_lines(), 4, 5)))
            #break0
            
            #for line2 in r.iter_lines():
            #    if 'STOPPED' in str(line2):
            #        print(line2)
            #print(line)
            #continue
    #test = next(itertools.islice(r.iter_lines(), 37, 38))
    #print(test)
    #if i==100:
    #    break
    

#for i in range
#print(next(r.iter_lines()))

#for line in r.iter_lines():
#    if line: print(line)
print(len(test))

In [ ]:
len(filenames)

In [ ]:
type(r.iter_lines().next())

#### Set up plotting function

In [ ]:
def show_image(file_number, frame_number):
    filename = filenames[file_number]
    plt.rc('figure', figsize=(10, 5))
    plt.rcParams.update({'font.size': 8})
    frame = camhd.get_frame(filename, frame_number, 'rgb24')
    fig, ax = plt.subplots();
    im1 = ax.imshow(frame);
    plt.yticks(np.arange(0,1081,270))
    plt.xticks(np.arange(0,1921,480))
    #plt.title(filename[84:]);
    plt.title(frame_number);

#### Interact

In [ ]:
from ipywidgets import interact
from ipywidgets import IntSlider
file_slider = IntSlider(min=0, max=len(filenames)-1, step=1, value=0, continuous_update=False)
frame_slider = IntSlider(min=0, max=camhd.get_frame_count(filenames[0]), step=1, value=0, continuous_update=False)

def update_range(*args):
    frame_slider.max = camhd.get_frame_count(filenames[file_slider.value])-1
    if frame_slider.value > frame_slider.max:
        frame_slider.value = frame_slider.max
file_slider.observe(update_range, 'value')

interact(show_image, file_number=file_slider, frame_number=frame_slider);

### References

PyCamHD: https://github.com/tjcrone/pycamhd<br>
CamHDHub: https://github.com/tjcrone/camhdhub<br>
Raw Data Archive: https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/<br>